# Initial QPE Circuit Baysian version.

In [50]:
class bayesian_qpe_builder():
    # Importing libraries.
    from pytket.circuit import Circuit
    from pytket.circuit import CircBox
    from pytket.circuit import QControlBox
    from pytket.circuit.display import render_circuit_jupyter
    
    def __init__(self, in_qubits, m_qubits, input_angle=1, z_angle=0, m=1):
        # Initializing the circuit.
        self.bayesian_qpe_circ: Circuit = Circuit()

        # Variables.
        self.m = m
        self.in_qubits = in_qubits
        self.m_qubits = m_qubits
        self.input_angle = input_angle
        self.z_angle = z_angle
        
        # Creating the initial qubits.
        initial_state = Circuit(in_qubits).X(0)  # Initial state of the circuit (|psi>).

        # Creating the unitary gate.
        unitary_control = Circuit(m_qubits).U1(input_angle, 0)  # Unitary control gate.
        unitary_control.name = "U"
        controlled_u_gate = QControlBox(CircBox(unitary_control), 1)
        
        # Initiate the measurement qubit.
        measurement_register = self.bayesian_qpe_circ.add_q_register("|0>", m_qubits)
        # Initiate the initial state.
        state_prep_register = self.bayesian_qpe_circ.add_q_register("psi", in_qubits)

        # Adding tha Hadamard gate.
        for m_qubit in measurement_register:
            self.bayesian_qpe_circ.H(m_qubit)

        # Build the circuit.
        self.bayesian_qpe_circ.add_circuit(initial_state, list(state_prep_register))
        
        # Adding the U-Control gate.
        for m_qubit in range(m_qubits):
            control_index = m_qubits - m_qubit - 1 # Contrtoling the index of the qubit.
            control_qubit = [measurement_register[control_index]] # Controling WHICH qubit.
            for _ in range(2**m):
                self.bayesian_qpe_circ.add_qcontrolbox(
                    controlled_u_gate, control_qubit + list(state_prep_register)
                )

        # Adding the Rz and Hadamard.
        for m_qubit in measurement_register:
            self.bayesian_qpe_circ.Rz(z_angle, m_qubit)

        for m_qubit in measurement_register:
            self.bayesian_qpe_circ.H(m_qubit)
        
        return render_circuit_jupyter(self.bayesian_qpe_circ)
            
        
bayesian_qpe = bayesian_qpe_builder(1, 1, m=0)